In [7]:
import speech_recognition as sr
import spacy
from astroquery.simbad import Simbad

NameError: name '_C' is not defined

In [ ]:


# Load the spaCy language model
nlp = spacy.load("custom_celestial_ner")

# Simbad query function to fetch celestial object coordinates
def fetch_celestial_object_coordinates(object_name):
    try:
        result_table = Simbad.query_object(object_name)
        if result_table:
            ra = result_table['RA'][0]  # Right Ascension
            dec = result_table['DEC'][0]  # Declination
            return {"RA": ra, "Dec": dec}
        else:
            return None
    except Exception as e:
        print(f"Error fetching data from Simbad: {e}")
        return None

# Function to parse the recognized command
def parse_command(command):
    doc = nlp(command)
    intent = None
    object_name = None
    
    # Extract intent based on keywords
    for token in doc:
        if token.lemma_ in ["find", "point", "show" ,"give"]:
            intent = token.lemma_
            
    # Extract potential celestial object name (proper nouns)
    for ent in doc.ents:
        if ent.label_ == "CELESTIAL_OBJECT":  
            object_name = ent.text
    
    return intent, object_name


In [ ]:
# Function to capture audio, recognize speech, and process commands
def listen_and_recognize():
    recognizer = sr.Recognizer()
    
    with sr.Microphone() as source:
        print("Adjusting for background noise...")
        recognizer.adjust_for_ambient_noise(source)
        print("Please speak for 5 seconds...")
        
        try:
            # Capture the audio for a maximum of 5 seconds
            audio = recognizer.listen(source, timeout=5, phrase_time_limit=5)
            print("Processing audio...")
            
            # Recognize the speech using Google Speech Recognition API
            command = recognizer.recognize_google(audio)
            print(f"Recognized command: {command}")
            
            # Parse the recognized text
            intent, object_name = parse_command(command)
            if intent and object_name:
                # Fetch celestial object coordinates dynamically from Simbad
                coordinates = fetch_celestial_object_coordinates(object_name)
                if coordinates:
                    print(f"Pointing to {object_name} at RA: {coordinates['RA']}, Dec: {coordinates['Dec']}")
                    # Send coordinates to the telescope controller (future step)
                else:
                    print(f"Sorry, {object_name} is not recognized as a celestial object in Simbad.")
            else:
                print("Command not understood.")
        
        except sr.WaitTimeoutError:
            print("Timeout: No speech detected in 5 seconds.")
        except sr.UnknownValueError:
            print("Sorry, I could not understand the audio.")
        except sr.RequestError as e:
            print(f"Could not request results from Google Speech Recognition service; {e}")



In [ ]:
if __name__ == "__main__":
    listen_and_recognize()

Adjusting for background noise...
Please speak for 5 seconds...
Processing audio...
Recognized command: find me m29
Pointing to m29 at RA: 20 23 46, Dec: +38 29.2


In [ ]:
from astropy.coordinates import EarthLocation, AltAz, SkyCoord
from astropy.time import Time
import astropy.units as u

# Set the observing location (latitude, longitude, and elevation in meters)
observing_location = EarthLocation(lat=18.5204*u.deg, lon=73.8567*u.deg, height=560*u.m)

# Set up the observing time to current time
observing_time = Time.now()

# Set the AltAz frame for the current location and time
altaz_frame = AltAz(location=observing_location, obstime=observing_time)

# Define the zenith position (altitude = 90 degrees, azimuth doesn't matter at zenith)
zenith = SkyCoord(alt=90*u.deg, az=0*u.deg, frame=altaz_frame)

# Convert to RA/Dec coordinates
zenith_radec = zenith.transform_to('icrs')

cur_ra = zenith_radec.ra.to_string(unit=u.hour)
cur_dec = zenith_radec.dec.to_string(unit=u.deg)

# Print the RA and Dec of the zenith
print(f"Current Zenith RA: {cur_ra}")
print(f"Current Zenith Dec: {cur_dec}")


Current Zenith RA: 17h57m29.04441761s
Current Zenith Dec: 18d31m18.18242687s
